The elo system for two players A and B with $E$ being the expected score (chance of winning in percent) and $R$ being the rating of the players says:
$$E_{A}={\frac {1}{1+10^{(R_{B}-R_{A})/400}}}$$
or conversely
$$E_{B}={\frac {1}{1+10^{(R_{A}-R_{B})/400}}}$$

Further, the adjustment after a match, with $R'$ being the updated rating, $S$ being the outcome, and $K$ being a scaling factor:
$$R_{A}^{\prime }=R_{A}+K(S_{A}-E_{A})$$
This scaling factor of $K$ represents the maximum the score can be adjusted by, and is apparently often adjusted depending on the player's current score. An example $K$ equation from the USCF, with $N_e$ being the effective number of games a rating is based on and $m$ being the number of games in a tournament:
$$K=800/(N_{e}+m)$$
Other implementations seem to be without formulas, e.g.:
$$K = \begin{cases}40 & \textrm{less than 30 games played as long as} R < 2300 \\ 20 & R \leq 2300 \\ 10 & R > 2300\end{cases}$$

In [72]:
def E(R):
    return (1 + 10 ** ((R[1] - R[0]) / 400)) ** -1

def Rn(R, S, K=None, N=1):
    if K is None:
        K = Kc(N, R[0])
    return R[0] + K * (S - E(R))

def Kc(N, R):
    if N < 30 and R < 2300:
        return 40
    elif R <= 2300:
        return 20
    else: # R > 2300
        return 10

An example wrapper:

In [73]:
all_players = {"Dellpit": [400, 1], "Eirox": [400, 1]}

In [80]:
def find_ratings(players, outcome):
    """
    Players will be saved in a dictionary. Outcome = 1 implies team 1 won and outcome = 2 implies team 2 won.
    """
    # S is winning team - 1
    S = outcome - 1
    
    p1 = all_players[players[0]]
    p2 = all_players[players[1]]
    
    R = [p1[0], p2[0]]
    N = [p1[1], p2[1]]
    
    Rnew = [Rn(R, S, N=N[0]), Rn(R[::-1], abs(S - 1), N=N[1])]
    all_players[players[0]] = [Rnew[0]
    all_players[players[1]] = Rnew[1]
    
    return Rnew

In [84]:
find_ratings(["Dellpit", "Eirox"], 1)

TypeError: 'float' object is not subscriptable

380.0
